<a href="https://colab.research.google.com/github/baua1/llm/blob/main/pre_training_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

import numpy as np
import pandas as pd

In [ ]:
class GPT(nn.Module):

  """
  cfg['vocab_size'] = the size if the vocabulary. That is how many tokens are present in the whole dataset.

  """

  def __init__(self, cfg):
    super().__init__()
    self.token_embeddings = nn.Embedding(cfg['vocab_size'], cfg['embd_dim'])
    self.pos_embeddings = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
    self.drop_embeddings = nn.Embedding(cfg['drop_rate'])

    self.transformer_block = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg['n_layers'])])

    self.final_norm = LayerNorm(cfg['embd_dim'])
    self.output_head = nn.Linear(cfg['embd_dim'], cfg['vocab_size'], bias=False)


  def forward(self,input_idx):

    batch_size, seq_len = input_idx.shape
    token_embeddings = self.token_embeddings(input_idx)
    position_embedding = self.pos_embeddings(torch.arange(seq_len, device=input_idx.device))
    x = token_embeddings + position_embedding
    x = self.drop(x)
    x = self.transformer_block(x)
    x = self.final_norm(x)
    logits = self.output_head(x)


    return logits


